In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ismailnasri20/driver-drowsiness-dataset-ddd")

print("Path to dataset files:", path)

100%|██████████| 2.58G/2.58G [00:29<00:00, 93.2MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/ismailnasri20/driver-drowsiness-dataset-ddd/versions/1


In [2]:
!pip install ultralytics kagglehub tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [3]:
import kagglehub
import os
import shutil
import random
from tqdm import tqdm
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
def download_and_prepare_data():
    # Download the dataset using KaggleHub
    path = kagglehub.dataset_download("ismailnasri20/driver-drowsiness-dataset-ddd")
    print(f"Dataset downloaded to: {path}")

    # Define the paths to drowsy and non-drowsy folders
    drowsy_folder = '/root/.cache/kagglehub/datasets/ismailnasri20/driver-drowsiness-dataset-ddd/versions/1/Driver Drowsiness Dataset (DDD)/Drowsy'
    non_drowsy_folder = '/root/.cache/kagglehub/datasets/ismailnasri20/driver-drowsiness-dataset-ddd/versions/1/Driver Drowsiness Dataset (DDD)/Non Drowsy'
    return drowsy_folder, non_drowsy_folder

In [6]:
dataset_path = os.path.join(path, "driver-drowsiness-dataset-ddd")

In [7]:
kaggle_drowsy_path = '/root/.cache/kagglehub/datasets/ismailnasri20/driver-drowsiness-dataset-ddd/versions/1/Driver Drowsiness Dataset (DDD)/Drowsy'
kaggle_non_drowsy_path = '/root/.cache/kagglehub/datasets/ismailnasri20/driver-drowsiness-dataset-ddd/versions/1/Driver Drowsiness Dataset (DDD)/Non Drowsy'
annotations_path = '/content/drive/MyDrive/drowsiness_detection/kaggle/annotations'
selected_images_path = '/content/drive/MyDrive/drowsiness_detection/kaggle/selected_images'
train_images_path = '/content/drive/MyDrive/drowsiness_detection/kaggle/train'
test_images_path = '/content/drive/MyDrive/drowsiness_detection/kaggle/test'

In [8]:
os.makedirs(annotations_path, exist_ok=True)
os.makedirs(selected_images_path, exist_ok=True)
os.makedirs(train_images_path, exist_ok=True)
os.makedirs(test_images_path, exist_ok=True)

In [9]:
def prepare_dataset(drowsy_folder, non_drowsy_folder, selected_images_folder):
    # Get all the image files in each folder
    drowsy_images = [os.path.join(drowsy_folder, img) for img in os.listdir(drowsy_folder) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]
    non_drowsy_images = [os.path.join(non_drowsy_folder, img) for img in os.listdir(non_drowsy_folder) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # Randomly select 750 images from each folder
    selected_drowsy = random.sample(drowsy_images, 750)
    selected_non_drowsy = random.sample(non_drowsy_images, 750)

    # Combine selected images into a single list
    selected_images = selected_drowsy + selected_non_drowsy

    # Copy selected images to the selected images folder
    if not os.path.exists(selected_images_folder):
        os.makedirs(selected_images_folder)

    for img_path in selected_images:
        shutil.copy(img_path, selected_images_folder)

    return selected_images_folder

In [10]:
import os
import cv2

def prepare_and_create_annotations(selected_images_folder, output_dir):
    """Generate annotations for selected images in a single function."""
    for img_name in os.listdir(selected_images_folder):
        if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):  # Check if the file is an image
            # Construct the image path
            img_path = os.path.join(selected_images_folder, img_name)

            # Define the label based on the folder (e.g., 'drowsy' or 'non_drowsy')
            label = 'drowsy' if 'drowsy' in img_name else 'non_drowsy'

            # Create annotation file for the image
            img = cv2.imread(img_path)
            height, width, _ = img.shape
            # For simplicity, we create a bounding box around the entire image
            x_center = 0.5  # Center of the image
            y_center = 0.5
            box_width = 1.0  # Full width
            box_height = 1.0  # Full height
            # Normalize bounding box coordinates
            annotation = f"{label} {x_center} {y_center} {box_width} {box_height}\n"

            # Write the annotation to the file
            filename = os.path.splitext(img_name)[0] + ".txt"
            annotation_path = os.path.join(output_dir, filename)
            with open(annotation_path, "w") as f:
                f.write(annotation)

            # Optionally, you can print to check progress
            print(f"Annotation saved for {img_name} at {annotation_path}")


In [11]:
def prepare_dataset(drowsy_folder, non_drowsy_folder, selected_images_folder):
    # Get all the image files in each folder
    drowsy_images = [os.path.join(drowsy_folder, img) for img in os.listdir(drowsy_folder) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]
    non_drowsy_images = [os.path.join(non_drowsy_folder, img) for img in os.listdir(non_drowsy_folder) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # Randomly select 750 images from each folder
    selected_drowsy = random.sample(drowsy_images, 750)
    selected_non_drowsy = random.sample(non_drowsy_images, 750)

    # Combine selected images into a single list
    selected_images = selected_drowsy + selected_non_drowsy

    # Copy selected images to the selected images folder
    if not os.path.exists(selected_images_folder):
        os.makedirs(selected_images_folder)

    for img_path in selected_images:
        shutil.copy(img_path, selected_images_folder)

    return selected_images_folder

In [12]:
from sklearn.model_selection import train_test_split

def split_dataset(selected_images_folder, test_size=0.2):
    image_paths = [os.path.join(selected_images_folder, img) for img in os.listdir(selected_images_folder)]
    labels = ['drowsy' if 'drowsy' in img else 'non_drowsy' for img in os.listdir(selected_images_folder)]

    # Split the dataset
    train_images, test_images = train_test_split(image_paths, test_size=test_size, random_state=42)

    return train_images, test_images

In [13]:
def copy_images_to_dirs(train_images, test_images, train_dir, test_dir):
    # Create directories if they do not exist
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    # Copy training images to train directory
    for img_path in train_images:
        shutil.copy(img_path, train_dir)

    # Copy testing images to test directory
    for img_path in test_images:
        shutil.copy(img_path, test_dir)

In [14]:
!pip install ultralytics
from ultralytics import YOLO

In [15]:
def train_yolo():
    """Train YOLOv8 model."""
    # Define the path to your YOLO data YAML file (YOLOv8)
    # This YAML file should describe your dataset (train/val paths, classes, etc.)
    data_yaml = '/content/drive/MyDrive/drowsiness_detection/kaggle/dataset.yaml'  # Path to your data YAML file


    # Define training parameters
    config = {
        'batch': 16,  # Batch size (correct argument name)
        'epochs': 100,  # Number of epochs
        'img_size': 640,  # Image size for training
        'weights': 'yolov8n.pt',  # Pretrained weights (you can change to another YOLOv8 model if needed)
        'project': '/content/drive/MyDrive/drowsiness_detection/kaggle/yolov8_training',  # Directory to save training results
        'name': 'drowsiness_detection',  # Name of the experiment
    }

    # Initialize the YOLOv8 model
    model = YOLO(config['weights'])  # Load the YOLOv8 model

    # Train the model with the updated configuration
    model.train(
        data=data_yaml,  # Use the data YAML file
        epochs=config['epochs'],
        imgsz=config['img_size'],
        batch=config['batch'],  # Use 'batch' instead of 'batch_size'
        project=config['project'],
        name=config['name']
    )

     # Evaluate the model on the validation set
    print("Evaluating YOLOv8 model on the test set...")
    results = model.val(data=data_yaml, imgsz=config['img_size'])

    # Display evaluation metrics
    print("\nEvaluation Metrics:")
    print(f"mAP@50: {results['metrics/mAP50']:.4f}")
    print(f"mAP@50-95: {results['metrics/mAP50-95']:.4f}")
    print(f"Precision: {results['metrics/precision']:.4f}")
    print(f"Recall: {results['metrics/recall']:.4f}")

In [16]:
import os

def fix_annotations(annotation_dir, class_index_map):
    """Fix annotation files by updating the class labels based on the first letter of the filename."""

    # Iterate through all annotation files in the directory
    for filename in os.listdir(annotation_dir):
        # Only process .txt files
        if filename.endswith(".txt"):
            file_path = os.path.join(annotation_dir, filename)

            # Check if the first character is uppercase or lowercase
            if filename[0].isupper():  # Starts with an uppercase letter means 'drowsy'
                correct_class = class_index_map['drowsy']
            elif filename[0].islower():  # Starts with a lowercase letter means 'non_drowsy'
                correct_class = class_index_map['non_drowsy']
            else:
                print(f"Skipping file {filename}: doesn't start with an alphabet letter.")
                continue  # Skip files that don't follow the naming convention

            # Open the annotation file and read its content
            with open(file_path, 'r') as file:
                lines = file.readlines()

            # Prepare a list to store the updated lines
            updated_lines = []
            changes_made = False

            # Process each line (each annotation)
            for line in lines:
                # Split the line into its components
                parts = line.strip().split()

                if len(parts) > 0:
                    # Update the class label with the correct class index
                    parts[0] = correct_class  # Replace with the correct class index
                    updated_lines.append(" ".join(parts) + "\n")
                    changes_made = True

            # If changes were made, write the updated annotations back to the file
            if changes_made:
                with open(file_path, 'w') as file:
                    file.writelines(updated_lines)
                print(f"Updated {filename}")
            else:
                print(f"No changes needed for {filename}")

# Define a map of old class labels to the new class indices
# Example: {'class_label': 'class_index'}
class_index_map = {
    'drowsy': '0',      # Replace 'drowsy' with 0
    'non_drowsy': '1',  # Replace 'non_drowsy' with 1
}

# Directory containing the annotation files
annotation_dir =  '/content/drive/MyDrive/drowsiness_detection/kaggle/train/labels' # Change this to your directory path

# Call the function to fix annotations
fix_annotations(annotation_dir, class_index_map)



Updated i0475.txt
Updated I0530.txt
Updated i0510.txt
Updated i0572.txt
Updated I0526.txt
Updated i0575.txt
Updated I0651.txt
Updated i0641.txt
Updated i0616.txt
Updated I0644.txt
Updated I0687.txt
Updated i0751.txt
Updated i0523.txt
Updated i0745.txt
Updated I0716.txt
Updated I0679.txt
Updated I0673.txt
Updated I0804.txt
Updated I0778.txt
Updated i0780.txt
Updated i0694.txt
Updated I0825.txt
Updated I0895.txt
Updated i0823.txt
Updated i0813.txt
Updated I0867.txt
Updated I0820.txt
Updated i0875.txt
Updated I0796.txt
Updated i0926.txt
Updated i0821.txt
Updated i0866.txt
Updated i0912.txt
Updated I0921.txt
Updated I0908.txt
Updated I1059.txt
Updated I0962.txt
Updated i0855.txt
Updated i0945.txt
Updated I1165.txt
Updated i1012.txt
Updated i0967.txt
Updated I1115.txt
Updated i1061.txt
Updated I1096.txt
Updated I1256.txt
Updated I1125.txt
Updated j0034.txt
Updated I1180.txt
Updated j0026.txt
Updated I1226.txt
Updated j0027.txt
Updated j0129.txt
Updated I1264.txt
Updated J0050(1).txt
Updated

In [ ]:
def run_pipeline():
    # Download and prepare data
    drowsy_folder, non_drowsy_folder = download_and_prepare_data()

    # Prepare the dataset by selecting 1500 images
    selected_images_folder = '/content/drive/MyDrive/drowsiness_detection/kaggle/selected_images'
    #selected_images_folder = prepare_dataset(drowsy_folder, non_drowsy_folder, selected_images_folder)

    # Prepare and create annotations for selected images
    #annotations_dir = '/content/drive/MyDrive/drowsiness_detection/kaggle/annotations'
    #prepare_and_create_annotations(selected_images_folder, annotations_dir)

    # Split dataset into train and test
    train_images, test_images = split_dataset(selected_images_folder)

    # Copy images to train/test directories
    # train_dir = '/content/drive/MyDrive/drowsiness_detection/kaggle/train'
    # test_dir = '/content/drive/MyDrive/drowsiness_detection/kaggle/test'
    # copy_images_to_dirs(train_images, test_images, train_dir, test_dir)

    # Train the YOLO model
    train_yolo()

In [ ]:
run_pipeline()

Dataset downloaded to: /root/.cache/kagglehub/datasets/ismailnasri20/driver-drowsiness-dataset-ddd/versions/1


100%|██████████| 6.25M/6.25M [00:00<00:00, 117MB/s]


Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/drowsiness_detection/kaggle/dataset.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/drowsiness_detection/kaggle/yolov8_training, name=drowsiness_detection9, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=Fals

100%|██████████| 755k/755k [00:00<00:00, 22.6MB/s]


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100      2.15G    0.02062    0.07614     0.8958         16        640: 100%|██████████| 75/75 [00:31<00:00,  2.39it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:03<00:00,  2.92it/s]

                   all        300        296      0.985          1       0.99       0.99



100 epochs completed in 1.044 hours.
Optimizer stripped from /content/drive/MyDrive/drowsiness_detection/kaggle/yolov8_training/drowsiness_detection9/weights/last.pt, 6.3MB
Optimizer stripped from /content/drive/MyDrive/drowsiness_detection/kaggle/yolov8_training/drowsiness_detection9/weights/best.pt, 6.3MB

Validating /content/drive/MyDrive/drowsiness_detection/kaggle/yolov8_training/drowsiness_detection9/weights/best.pt...
Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  50%|█████     | 5/10 [00:04<00:05,  1.18s/it]

In [ ]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import os

# Load the YOLO model
model_path = '/content/drive/MyDrive/drowsiness_detection/kaggle/yolov8_training/drowsiness_detection9/weights/best.pt'
model = YOLO(model_path)


In [ ]:
def visualize_results(model, test_images_dir, annotation_dir, num_samples=5):
    """
    Visualize predictions on the test dataset.

    Args:
        model: Trained YOLO model.
        test_images_dir: Path to test images directory.
        annotation_dir: Path to annotation files directory (optional for comparison).
        num_samples: Number of samples to visualize.
    """
    # Get test image file paths
    image_files = [os.path.join(test_images_dir, f) for f in os.listdir(test_images_dir) if f.endswith(('.jpg', '.png'))]
    image_files = sorted(image_files)[:num_samples]

    for img_path in image_files:
        # Load image
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Get predictions
        results = model(img_path)

        # Extract bounding boxes, class indices, and confidences
        predictions = results[0].boxes
        if predictions is not None:
            for box, cls, conf in zip(predictions.xyxy, predictions.cls, predictions.conf):
                # Draw bounding box
                x1, y1, x2, y2 = map(int, box)
                class_label = int(cls)
                label = f"{model.names[class_label]}: {conf:.2f}"
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        # Display image with predictions
        plt.figure(figsize=(10, 10))
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"Predictions for {os.path.basename(img_path)}")
        plt.show()


In [ ]:
# Paths to test images and annotations
test_images_dir = '/content/drive/MyDrive/drowsiness_detection/kaggle/test/images'
annotation_dir = '/content/drive/MyDrive/drowsiness_detection/kaggle/test/labels'

# Visualize results on test images
visualize_results(model, test_images_dir, annotation_dir, num_samples=5)


NameError: name 'visualize_results' is not defined

In [ ]:
import cv2
import os
import random
import ultralytics

# Load the YOLOv8 model
model_path = '/content/drive/MyDrive/drowsiness_detection/kaggle/yolov8_training/drowsiness_detection9/weights/best.pt' # Path to your trained YOLOv8 model
model = ultralytics.YOLO(model_path)

# Define paths
test_images_dir = "/content/drive/MyDrive/drowsiness_detection/kaggle/test/images"
output_visualizations_dir = "/content/detection_visualizations"  # Output directory for visualizations
os.makedirs(output_visualizations_dir, exist_ok=True)

# Evaluate model on the test set to get metrics
results = model.val(data="/content/drive/MyDrive/drowsiness_detection/kaggle/dataset.yaml", split="val")
metrics = results.metrics  # Retrieve the evaluation metrics

# Extract specific metrics
precision = metrics["precision"]
recall = metrics["recall"]
map50 = metrics["map50"]
map5095 = metrics["map"]  # mAP@50-95

# Select 20 random images
image_files = os.listdir(test_images_dir)
selected_images = random.sample(image_files, min(20, len(image_files)))

# Visualize predictions with metrics
for img_file in selected_images:
    img_path = os.path.join(test_images_dir, img_file)

    # Load image
    img = cv2.imread(img_path)
    if img is None:
        continue

    # Get predictions for the image
    results = model.predict(source=img_path, conf=0.25, save=False)
    detections = results[0].boxes

    # Draw predictions on the image
    for box in detections:
        x_min, y_min, x_max, y_max = map(int, box.xyxy[0].tolist())
        confidence = box.conf[0]
        class_id = int(box.cls[0])
        label = "Drowsy" if class_id == 0 else "Non-Drowsy"
        cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
        cv2.putText(img, f"Pred: {label} ({confidence:.2f})", (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    # Overlay general metrics
    metrics_text = (
        f"Precision: {precision:.3f}\n"
        f"Recall: {recall:.3f}\n"
        f"mAP@50: {map50:.3f}\n"
        f"mAP@50-95: {map5095:.3f}"
    )
    y_offset = 20
    for i, line in enumerate(metrics_text.split("\n")):
        cv2.putText(img, line, (10, y_offset + i * 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)

    # Save visualization
    output_path = os.path.join(output_visualizations_dir, img_file)
    cv2.imwrite(output_path, img)
    print(f"Visualization saved: {output_path}")


Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 168 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/drive/MyDrive/drowsiness_detection/kaggle/test/labels.cache... 296 images, 4 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  68%|██████▊   | 13/19 [01:22<00:38,  6.36s/it]


KeyboardInterrupt: 

In [ ]:
import os
import cv2
import random
from pathlib import Path
from ultralytics import YOLO

# Load your trained YOLOv8 model
model_path = '/content/drive/MyDrive/drowsiness_detection/kaggle/yolov8_training/drowsiness_detection9/weights/best.pt'
model = YOLO(model_path)

# Define paths
test_images_dir = "/content/drive/MyDrive/drowsiness_detection/kaggle/test/images"
output_visualizations_dir = "/content/detection_visualizations"
os.makedirs(output_visualizations_dir, exist_ok=True)

# Evaluate the model to get overall metrics
results = model.val(data="/content/drive/MyDrive/drowsiness_detection/kaggle/dataset.yaml", split="val")

# Extract overall metrics
precision = results.box.maps['precision']
recall = results.box.maps['recall']
map50 = results.box.maps[0]
map5095 = results.box.maps.mean()

# Format the metrics as a string for overlay
metrics_text = (f"Precision: {precision:.3f}, Recall: {recall:.3f}, "
                f"mAP@50: {map50:.3f}, mAP@50-95: {map5095:.3f}")

# Select 20 random test images
image_files = list(Path(test_images_dir).glob("*.jpg"))
selected_images = random.sample(image_files, min(20, len(image_files)))

# Loop through the selected images
for img_path in selected_images:
    img_path = str(img_path)
    img = cv2.imread(img_path)

    # Get predictions
    preds = model.predict(source=img_path, conf=0.25, save=False)
    detections = preds[0].boxes

    # Draw predictions on the image
    for box in detections:
        x_min, y_min, x_max, y_max = map(int, box.xyxy[0].tolist())
        confidence = box.conf[0]
        class_id = int(box.cls[0])
        label = model.names[class_id]

        # Draw the bounding box and label
        cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
        cv2.putText(img, f"{label} ({confidence:.2f})", (x_min, y_min - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    # Overlay the overall metrics on the image
    y_start = 30  # Starting y-coordinate for text
    for line in metrics_text.split(", "):
        cv2.putText(img, line, (10, y_start), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
        y_start += 20

    # Save the visualized image
    output_path = f"{output_visualizations_dir}/{Path(img_path).name}"
    cv2.imwrite(output_path, img)
    print(f"Saved visualization: {output_path}")


Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 168 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/drive/MyDrive/drowsiness_detection/kaggle/test/labels.cache... 296 images, 4 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [02:03<00:00,  6.51s/it]


                   all        300        296      0.982          1      0.993      0.993
                drowsy        142        142       0.97          1      0.991      0.991
            non_drowsy        154        154      0.993          1      0.995      0.995
Speed: 7.5ms preprocess, 378.5ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs/detect/val3


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
import os
import cv2
import random
from pathlib import Path
from ultralytics import YOLO

# Load your trained YOLOv8 model
model_path = '/content/drive/MyDrive/drowsiness_detection/kaggle/yolov8_training/drowsiness_detection9/weights/best.pt'
model = YOLO(model_path)

# Define paths
test_images_dir = "/content/drive/MyDrive/drowsiness_detection/kaggle/test/images"
output_visualizations_dir = "/content/detection_visualizations"
os.makedirs(output_visualizations_dir, exist_ok=True)

# Evaluate the model to get metrics
results = model.val(data="/content/drive/MyDrive/drowsiness_detection/kaggle/dataset.yaml", split="val")

# Extract overall metrics
metrics_summary = results.results_dict  # Get metrics summary as a dictionary
precision = metrics_summary['metrics/precision(B)']  # Precision
recall = metrics_summary['metrics/recall(B)']        # Recall
map50 = metrics_summary['metrics/mAP50(B)']          # mAP@50
map5095 = metrics_summary['metrics/mAP50-95(B)']     # mAP@50-95

# Format the metrics as a string for overlay
metrics_text = (f"Precision: {precision:.3f}, Recall: {recall:.3f}, "
                f"mAP@50: {map50:.3f}, mAP@50-95: {map5095:.3f}")

# Select 20 random test images
image_files = list(Path(test_images_dir).glob("*.jpg"))
selected_images = random.sample(image_files, min(20, len(image_files)))

# Loop through the selected images
for img_path in selected_images:
    img_path = str(img_path)
    img = cv2.imread(img_path)

    # Get predictions
    preds = model.predict(source=img_path, conf=0.25, save=False)
    detections = preds[0].boxes

    # Draw predictions on the image
    for box in detections:
        x_min, y_min, x_max, y_max = map(int, box.xyxy[0].tolist())
        confidence = box.conf[0]
        class_id = int(box.cls[0])
        label = model.names[class_id]

        # Draw the bounding box and label
        cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
        cv2.putText(img, f"{label} ({confidence:.2f})", (x_min, y_min - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    # Overlay the overall metrics on the image
    y_start = 30  # Starting y-coordinate for text
    for line in metrics_text.split(", "):
        cv2.putText(img, line, (10, y_start), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
        y_start += 20

    # Save the visualized image
    output_path = f"{output_visualizations_dir}/{Path(img_path).name}"
    cv2.imwrite(output_path, img)
    print(f"Saved visualization: {output_path}")


Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 168 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


100%|██████████| 755k/755k [00:00<00:00, 61.4MB/s]
val: Scanning /content/drive/MyDrive/drowsiness_detection/kaggle/test/labels.cache... 296 images, 4 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [01:48<00:00,  5.69s/it]


                   all        300        296      0.982          1      0.993      0.993
                drowsy        142        142       0.97          1      0.991      0.991
            non_drowsy        154        154      0.993          1      0.995      0.995
Speed: 8.3ms preprocess, 321.7ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/val


In [ ]:
import os
import random
from pathlib import Path
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO

# Load your trained YOLOv8 model
model_path = '/content/drive/MyDrive/drowsiness_detection/kaggle/yolov8_training/drowsiness_detection9/weights/best.pt'
model = YOLO(model_path)

# Define test image directory
test_images_dir = "/content/drive/MyDrive/drowsiness_detection/kaggle/test/images"

# Select 20 random test images
image_files = list(Path(test_images_dir).glob("*.jpg"))
selected_images = random.sample(image_files, min(20, len(image_files)))

# Loop through the selected images and visualize results
for img_path in selected_images:
    img_path = str(img_path)
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for matplotlib

    # Get predictions
    preds = model.predict(source=img_path, conf=0.25, save=False)
    detections = preds[0].boxes

    # Draw predictions on the image
    for box in detections:
        x_min, y_min, x_max, y_max = map(int, box.xyxy[0].tolist())
        confidence = box.conf[0]
        class_id = int(box.cls[0])
        label = model.names[class_id]

        # Draw the bounding box and label
        color = (0, 255, 0) if label == "non_drowsy" else (0, 0, 255)  # Green for non-drowsy, red for drowsy
        cv2.rectangle(img_rgb, (x_min, y_min), (x_max, y_max), color, 2)
        cv2.putText(img_rgb, f"{label} ({confidence:.2f})", (x_min, y_min - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

    # Show the image with detections
    plt.figure(figsize=(8, 8))
    plt.imshow(img_rgb)
    plt.title(f"Detections for {Path(img_path).name}")
    plt.axis("off")
    plt.show()


In [ ]:
import os
import random
from pathlib import Path
import cv2
from ultralytics import YOLO

# Load your trained YOLOv8 model
model_path = '/content/drive/MyDrive/drowsiness_detection/kaggle/yolov8_training/drowsiness_detection9/weights/best.pt'

# Check if the model file exists
if not os.path.exists(model_path):
    print(f"Error: Model file not found at {model_path}")
    exit()

model = YOLO(model_path)
print("Model loaded successfully.")

# Define paths
test_images_dir = "/content/drive/MyDrive/drowsiness_detection/kaggle/test/images"

# Check if test directory exists
if not os.path.exists(test_images_dir):
    print(f"Error: Test images directory not found at {test_images_dir}")
    exit()

# Select 20 random test images
image_files = list(Path(test_images_dir).glob("*.jpg"))
if len(image_files) == 0:
    print("No images found in the test directory.")
    exit()

selected_images = random.sample(image_files, min(20, len(image_files)))
print(f"Selected {len(selected_images)} images for testing.")

# Loop through the selected images and make predictions
for img_path in selected_images:
    img_path = str(img_path)
    img = cv2.imread(img_path)

    # Check if the image was loaded correctly
    if img is None:
        print(f"Error: Unable to load image {img_path}")
        continue

    # Get predictions
    preds = model.predict(source=img_path, conf=0.25, save=False)

    # Check if predictions are valid
    if preds is None or len(preds) == 0:
        print(f"No predictions for image: {img_path}")
        continue

    detections = preds[0].boxes

    # Prepare the list to store labels for each image
    detected_labels = []

    # Check detections and store labels for "drowsy" and "non_drowsy"
    for box in detections:
        class_id = int(box.cls[0])
        label = model.names[class_id]

        # Only store labels for "drowsy" and "non_drowsy"
        if label in ["drowsy", "non_drowsy"]:
            detected_labels.append(label)

    # Print the detected labels for the current image
    if detected_labels:
        print(f"Image: {img_path} -> Detected Labels: {', '.join(detected_labels)}")
    else:
        print(f"Image: {img_path} -> No Drowsy/Non-Drowsy Detected")


Model loaded successfully.
No images found in the test directory.
Selected 0 images for testing.
